app: 2019-11-16 to 
web: 2019-11-25 to 2019-12-01


In [3]:
import warnings
warnings.filterwarnings('ignore',)

import importlib
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#Initialize Package
import pmutils.creds as pu
from pmstats.abtest import *
from pmstats.plots import *
from pmstats.lightspeed import *
from pmstats.utils import *

#Other packages
import pandas as pd
import numpy as np

from datetime import date, timedelta

In [4]:
my_conn = pu.create_connection()

In [5]:
#versions=['3.22','3.22.01','3.22.02']
versions=None
print(versions)
test_segments = ['G','J']
control_segments = ['E','I']

pre_start_date = '2019-10-29'
pre_end_date = '2019-11-11'
post_start_date = '2019-11-16'
post_start_date_web = '2019-11-25'
post_end_date = '2019-12-15'
#post_end_date = '2019-12-01'

None


In [6]:
north_star_metrics = ['orders_buy_now_200andmore'
    ,'gmv_buy_now_200andmore'
    ,'aov_buy_now_200andmore'
]

business_metrics = ['gmv'
    ,'orders'
    ,'aov'
    ,'buyers'
    ,'gmv - buy now'
    ,'orders - buy now'
    ,'aov - buy now'
    ,'buyers - buy now'
    ,'gmv_200andmore'
    ,'orders_200andmore'
]

business_metrics_other = ['gmv - seller offer'
    ,'orders - seller offer'
    ,'buyers - seller offer'
    ,'gmv - buyer offer'
    ,'orders - buyer offer'
    ,'buyers - buyer offer']

feature_metrics = ['buy now clicks'
    ,'buy now clickers'
    ,'buy confirmation views'
    ,'buy confirmation viewers'
    ,'submit order clicks'
    ,'submit order clickers'
]


other_metrics = ['gmv_buy_now_200_350'
    ,'orders_buy_now_200_350'
    ,'gmv_buy_now_350_500'
    ,'orders_buy_now_350_500'
    ,'gmv_buy_now_500_1000'
    ,'orders_buy_now_500_1000'
    ,'gmv_buy_now_1000andmore'
    ,'orders_buy_now_1000andmore'
]

test_metrics = north_star_metrics + business_metrics + feature_metrics + business_metrics_other

In [9]:
luxury_orders = dict(
    name = "luxury_orders",
    sql = """
    SELECT
        buyer_id as user_id
        , booked_at::date as event_date
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=200 THEN order_gmv*0.01 END) as gmv_buy_now_200
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=200 THEN 1 END) as orders_buy_now_200
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=200 AND dw_orders.order_gmv*0.01<350 THEN order_gmv*0.01 END) as gmv_buy_now_200_350
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=200 AND dw_orders.order_gmv*0.01<350 THEN 1 END) as orders_buy_now_200_350
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=350 AND dw_orders.order_gmv*0.01<500 THEN order_gmv*0.01 END) as gmv_buy_now_350_500
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=350 AND dw_orders.order_gmv*0.01<500 THEN 1 END) as orders_buy_now_350_500
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=500 AND dw_orders.order_gmv*0.01<1000 THEN order_gmv*0.01 END) as gmv_buy_now_500_1000
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=500 AND dw_orders.order_gmv*0.01<1000 THEN 1 END) as orders_buy_now_500_1000
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=1000 THEN order_gmv*0.01 END) as gmv_buy_now_1000
        , SUM(CASE WHEN offer_id is null AND dw_orders.order_gmv*0.01>=1000 THEN 1 END) as orders_buy_now_1000
        , SUM(CASE WHEN dw_orders.order_gmv*0.01>=200 THEN order_gmv*0.01 END) as gmv_200
        , SUM(CASE WHEN dw_orders.order_gmv*0.01>=200 THEN 1 END) as orders_200
FROM analytics.dw_orders
    WHERE booked_at::date BETWEEN '{metric_start_date}' AND '{metric_end_date}'
    AND app IN ({metric_activity_apps})
    GROUP BY 1,2
    """,
    join = "luxury_orders.user_id = exposure.user_id AND luxury_orders.event_date = exposure.event_date",
    metric_list = [
        "SUM(gmv_buy_now_200) as gmv_buy_now_200andmore",
        "SUM(orders_buy_now_200) as orders_buy_now_200andmore",
        "1.0*SUM(gmv_buy_now_200)/SUM(orders_buy_now_200) as aov_buy_now_200andmore",
        "SUM(gmv_buy_now_200_350) as gmv_buy_now_200_350",
        "SUM(orders_buy_now_200_350) as orders_buy_now_200_350",
        "SUM(gmv_buy_now_350_500) as gmv_buy_now_350_500",
        "SUM(orders_buy_now_350_500) as orders_buy_now_350_500",
        "SUM(gmv_buy_now_500_1000) as gmv_buy_now_500_1000",
        "SUM(orders_buy_now_500_1000) as orders_buy_now_500_1000", 
        "SUM(gmv_buy_now_1000) as gmv_buy_now_1000andmore",
        "SUM(orders_buy_now_1000) as orders_buy_now_1000andmore",
        "SUM(gmv_200) as gmv_200andmore",
        "SUM(orders_200) as orders_200andmore",
        ]
    )

In [10]:
#all 3 platforms
pre_all, post_all, _, _ = create_prepost(conn=my_conn,
                 control_segments=control_segments,
                 test_segments=test_segments,
                 pre_start_date = pre_start_date,
                 pre_end_date = pre_end_date,
                 post_start_date = post_start_date_web,
                 post_end_date = post_end_date,
                 post_exposure_versions = versions, 
                 segment_type='signup_segment',
                 reg_apps=['iphone','android','web'],
                 exposure_activity_apps= ['iphone','android','web'],
                 metric_activity_apps=['iphone','android','web'],
                 standard_metrics=['BUYER_ORDERS','BUYER_OFFERS'],
                 adhoc_metrics=[luxury_orders]
                                              )


Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers
Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers


In [11]:
ab_result_all=create_pretty_outputs(pre_all, 
                                    post_all, 
                                    max_pvalue=0.2,
                                    title='Affirm AB Test All Platforms',
                                    lightspeed_only=True, 
                                    must_include_metrics=test_metrics)

,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Daily Impact
0,buyers,"14,580.36","14,299.57",-1.91%,"13,742.33","13,502.71",-1.69%,0.23%,-0.46%,0.91%,0.5065,31.024
2,orders,"19,199.14","18,767.64",-2.28%,"17,761.19","17,425.81",-1.84%,0.44%,-0.26%,1.13%,0.2076,77.475
7,gmv,"657,195.18","635,296.61",-3.29%,"648,952.50","645,786.66",-0.45%,2.84%,1.39%,4.29%,0.0004,"18,438.542"
15,orders - buyer offer,"8,638.29","8,465.79",-2.01%,"7,597.67","7,440.76",-2.04%,-0.03%,-1.44%,1.37%,0.9612,-2.507
16,gmv - buyer offer,"324,257.21","311,898.50",-3.76%,"302,976.86","293,816.86",-2.94%,0.83%,-1.61%,3.26%,0.4949,"2,501.218"
19,orders - seller offer,"4,862.71","4,773.29",-1.87%,"4,262.19","4,221.57",-0.92%,0.95%,-0.48%,2.39%,0.1837,40.594
22,gmv - seller offer,"143,004.36","139,305.32",-2.52%,"127,030.19","126,850.66",-0.12%,2.39%,-0.50%,5.29%,0.1015,"3,042.301"
24,orders - buy now,"5,698.14","5,528.57",-2.97%,"5,901.33","5,763.48",-2.25%,0.72%,-0.69%,2.12%,0.3037,42.251
30,gmv - buy now,"189,933.61","184,092.79",-2.84%,"218,945.45","225,119.15",3.05%,5.89%,2.53%,9.25%,0.0014,"12,900.741"
44,gmv_buy_now_200andmore,"36,728.84","35,287.62",-1.40%,"50,166.87","60,864.70",23.36%,24.76%,10.92%,38.60%,0.0011,"12,421.734"


In [12]:
ab_result_all_summary = create_pretty_outputs(pre_all, 
                        post_all, 
                        title='Affirm AB Test All Platforms', 
                        lightspeed_only=True,
                        must_include_metrics =test_metrics,
                        diagnostic=False,
                        max_pvalue=0.2,
                        stats_list = ['Metric',
                                      #'Pre Control Metric Mean',
                                      #'Pre Test Metric Mean',
                                      #'Pre Delta (%)',
                                      #'Post Control Metric Mean',
                                      #'Post Test Metric Mean',
                                      #'Post Delta (%)',
                                      'PrePost Delta (%)',
                                      #'PrePost Delta LCL (%)',
                                      #'PrePost Delta UCL (%)',
                                      'p-value',
                                      #'Net Daily Impact',
                                     ])

,Metric,PrePost Delta (%),p-value
0,buyers,0.23%,0.5065
2,orders,0.44%,0.2076
7,gmv,2.84%,0.0004
15,orders - buyer offer,-0.03%,0.9612
16,gmv - buyer offer,0.83%,0.4949
19,orders - seller offer,0.95%,0.1837
22,gmv - seller offer,2.39%,0.1015
24,orders - buy now,0.72%,0.3037
30,gmv - buy now,5.89%,0.0014
44,gmv_buy_now_200andmore,24.76%,0.0011


In [13]:
#price breakdown
ab_result_all=create_pretty_outputs(pre_all, 
                                    post_all, 
                                    max_pvalue=0.2,
                                    title='Affirm AB Test All Platforms',
                                    lightspeed_only=True, 
                                    must_include_metrics=other_metrics,
                                    must_exclude_metrics=test_metrics)


,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Daily Impact
47,gmv_buy_now_200_350,"11,268.95","11,510.61",4.23%,"16,748.72","18,716.60",14.06%,9.83%,-4.09%,23.76%,0.1588,"1,646.968"
48,orders_buy_now_200_350,45.43,46.14,3.28%,67.24,74.71,13.33%,10.05%,-2.43%,22.53%,0.1105,6.756
49,gmv_buy_now_350_500,"5,881.03","5,308.45",5.25%,"8,511.52","11,727.67",47.44%,42.19%,1.62%,82.76%,0.0422,"3,591.012"
50,orders_buy_now_350_500,14.71,13.07,2.96%,21.10,28.76,45.55%,42.58%,4.47%,80.69%,0.0300,8.983
51,gmv_buy_now_500_1000,"9,818.39","9,354.44",-1.80%,"13,058.44","17,230.49",38.88%,40.68%,15.67%,65.69%,0.0023,"5,311.998"
52,orders_buy_now_500_1000,14.93,14.00,-3.74%,20.14,25.62,33.55%,37.29%,13.61%,60.97%,0.0030,7.512
53,gmv_buy_now_1000andmore,"9,760.46","9,114.12",13.19%,"11,848.19","13,189.95",58.01%,44.82%,-39.80%,129.44%,0.2879,"5,310.473"
54,orders_buy_now_1000andmore,6.00,5.93,10.45%,6.67,8.33,59.96%,49.51%,-15.94%,114.96%,0.1330,3.301
57,gmv per orders,34.26,33.90,-1.03%,36.57,37.08,1.42%,2.45%,1.05%,3.85%,0.0014,0.895


In [12]:
#app
pre_app, post_app, _, _ = create_prepost(conn=my_conn,
                 control_segments=control_segments,
                 test_segments=test_segments,
                 pre_start_date = pre_start_date,
                 pre_end_date = pre_end_date,
                 post_start_date = post_start_date,
                 post_end_date = post_end_date,
                 post_exposure_versions = versions, 
                 segment_type='signup_segment',
                 reg_apps=['iphone','android'],
                 exposure_activity_apps= ['iphone','android'],
                 metric_activity_apps=['iphone','android'],
                 standard_metrics=['BUYER_ORDERS','BUYER_OFFERS'],
                 adhoc_metrics=[luxury_orders]
                                              )



Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers
Ratio Error: gmv per dau
Ratio Error: buyers per dau
Ratio Error: listers per dau
Ratio Error: listings per dau
Ratio Error: listings per listers


In [16]:
ab_result_app=create_pretty_outputs(pre_app, 
                                    post_app, 
                                    max_pvalue=0.2,
                                    title='Affirm AB Test Android and iPhone',
                                    lightspeed_only=True, 
                                    must_include_metrics=test_metrics)

,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Daily Impact
0,buyers,"11,692.21","11,450.86",-2.03%,"11,141.94","11,016.38",-1.08%,0.95%,0.09%,1.81%,0.0311,105.851
2,orders,"15,430.93","15,047.79",-2.52%,"14,574.69","14,425.75",-0.98%,1.54%,0.53%,2.55%,0.0046,224.244
5,gmv,"517,254.33","496,976.68",-3.83%,"499,087.12","495,300.81",-0.70%,3.13%,0.98%,5.28%,0.0058,"15,623.320"
12,orders - buyer offer,"7,301.50","7,137.93",-2.26%,"6,743.56","6,713.56",-0.46%,1.80%,0.18%,3.41%,0.0305,121.172
13,gmv - buyer offer,"272,119.50","261,137.00",-3.95%,"259,455.38","252,104.19",-2.77%,1.17%,-1.65%,3.99%,0.4007,"3,047.137"
16,orders - seller offer,"4,297.86","4,205.36",-2.17%,"4,096.25","4,053.50",-0.99%,1.17%,-0.37%,2.72%,0.1297,48.110
18,gmv - seller offer,"126,763.86","123,048.39",-2.82%,"120,532.50","120,115.50",-0.21%,2.61%,-0.65%,5.88%,0.1124,"3,149.629"
19,orders - buy now,"3,831.57","3,704.50",-3.37%,"3,734.88","3,658.69",-1.89%,1.48%,-0.69%,3.65%,0.1742,55.188
25,gmv - buy now,"118,370.97","112,791.29",-4.27%,"119,099.25","123,081.12",3.46%,7.73%,2.80%,12.65%,0.0033,"9,203.129"
38,gmv_buy_now_200andmore,"22,353.40","20,583.55",-3.00%,"25,316.30","30,434.73",23.93%,26.93%,5.86%,48.01%,0.0141,"6,817.805"


In [25]:
ab_result_app_summary = create_pretty_outputs(pre_app, 
                        post_app, 
                        title='Affirm AB Test Android and iPhone', 
                        lightspeed_only=True,
                        must_include_metrics =test_metrics,
                        diagnostic=False,
                        max_pvalue=0.2,
                        stats_list = ['Metric',
                                      #'Pre Control Metric Mean',
                                      #'Pre Test Metric Mean',
                                      #'Pre Delta (%)',
                                      #'Post Control Metric Mean',
                                      #'Post Test Metric Mean',
                                      #'Post Delta (%)',
                                      'PrePost Delta (%)',
                                      #'PrePost Delta LCL (%)',
                                      #'PrePost Delta UCL (%)',
                                      'p-value',
                                      #'Net Daily Impact',
                                     ])

,Metric,PrePost Delta (%),p-value
0,buyers,0.95%,0.0311
2,orders,1.54%,0.0046
5,gmv,3.13%,0.0058
19,orders - buy now,1.48%,0.1742
25,gmv - buy now,7.73%,0.0033
38,gmv_buy_now_200andmore,26.93%,0.0141
39,orders_buy_now_200andmore,21.75%,0.0035
40,aov_buy_now_200andmore,5.66%,0.4208
49,gmv per orders,1.62%,0.1065


In [17]:
ab_result_app=create_pretty_outputs(pre_app, 
                                    post_app, 
                                    max_pvalue=0.2,
                                    title='Affirm AB Test Android and iPhone',
                                    lightspeed_only=True, 
                                    must_include_metrics=other_metrics,
                                    must_exclude_metrics=test_metrics)


,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Daily Impact
41,gmv_buy_now_200_350,"6,828.18","6,599.19",1.94%,"7,343.07","8,692.82",18.75%,16.80%,-2.87%,36.47%,0.0911,"1,233.806"
42,orders_buy_now_200_350,27.14,26.36,1.18%,29.50,34.62,17.77%,16.59%,-0.52%,33.71%,0.0569,4.895
43,gmv_buy_now_350_500,"3,578.51","3,120.95",7.69%,"4,033.52","4,739.88",34.62%,26.94%,-23.25%,77.12%,0.2795,"1,086.540"
44,orders_buy_now_350_500,8.93,7.64,3.99%,10.00,11.62,30.09%,26.10%,-18.39%,70.59%,0.2380,2.610
45,gmv_buy_now_500_1000,"5,419.49","5,641.44",12.26%,"7,517.37","9,678.94",41.33%,29.07%,-4.02%,62.17%,0.0827,"2,185.581"
46,orders_buy_now_500_1000,8.14,8.36,8.04%,11.50,14.50,39.72%,31.67%,0.73%,62.62%,0.0452,3.642
47,gmv_buy_now_1000andmore,"6,527.21","5,221.96",13.86%,"6,850.50","7,323.09",nan%,nan%,nan%,nan%,nan,nan
48,orders_buy_now_1000andmore,3.79,3.57,10.32%,3.60,4.50,nan%,nan%,nan%,nan%,nan,nan
49,gmv per orders,33.56,33.10,-1.34%,34.30,34.38,0.28%,1.62%,-0.37%,3.62%,0.1065,0.557


In [6]:
ab_result_app_summary = create_pretty_outputs(pre_app, 
                        post_app, 
                        title='Affirm AB Test Android and iPhone', 
                        lightspeed_only=True,
                        must_include_metrics =other_metrics,
                        must_exclude_metrics=test_metrics,
                        diagnostic=False,
                        max_pvalue=0.2,
                        stats_list = ['Metric',
                                      #'Pre Control Metric Mean',
                                      #'Pre Test Metric Mean',
                                      #'Pre Delta (%)',
                                      #'Post Control Metric Mean',
                                      #'Post Test Metric Mean',
                                      #'Post Delta (%)',
                                      'PrePost Delta (%)',
                                      #'PrePost Delta LCL (%)',
                                      #'PrePost Delta UCL (%)',
                                      'p-value',
                                      'Net Daily Impact',
                                     ])

NameError: name 'pre_app' is not defined

In [7]:
#web

pre_web, post_web, _, _ = create_prepost(conn=my_conn,
                 control_segments=control_segments,
                 test_segments=test_segments,
                 pre_start_date = pre_start_date,
                 pre_end_date = pre_end_date,
                 post_start_date = post_start_date_web,
                 post_end_date = post_end_date,
                 post_exposure_versions = versions, 
                 segment_type='signup_segment',
                 reg_apps=['web'],
                 exposure_activity_apps= ['web'],
                 metric_activity_apps=['web'],
                 standard_metrics=['BUYER_ORDERS','BUYER_OFFERS'],
                 adhoc_metrics=[luxury_orders]
                                              )



KeyboardInterrupt: 

In [22]:
ab_result_web=create_pretty_outputs(pre_web, 
                                    post_web, 
                                    max_pvalue=0.2,
                                    title='Affirm AB Test Web',
                                    lightspeed_only=True, 
                                    must_include_metrics=test_metrics)

,Metric,Pre Control Metric Mean,Pre Test Metric Mean,Pre Delta (%),Post Control Metric Mean,Post Test Metric Mean,Post Delta (%),PrePost Delta (%),PrePost Delta LCL (%),PrePost Delta UCL (%),p-value,Net Daily Impact
0,buyers,"2,977.50","2,937.07",-1.42%,"2,806.86","2,744.43",-2.16%,-0.73%,-2.55%,1.08%,0.3979,-20.612
2,orders,"3,759.71","3,709.00",-1.38%,"3,478.29","3,396.86",-2.17%,-0.79%,-3.43%,1.85%,0.5243,-27.506
5,gmv,"139,717.49","137,812.93",-1.32%,"141,298.89","140,767.26",-0.11%,1.21%,-2.62%,5.05%,0.5125,"1,716.093"
12,orders - buyer offer,"1,332.50","1,322.71",-0.66%,"1,076.00","1,040.86",-3.31%,-2.64%,-7.80%,2.52%,0.2909,-28.434
13,gmv - buyer offer,"52,037.43","50,553.14",-2.46%,"44,809.71","42,814.57",-4.51%,-2.05%,-8.96%,4.86%,0.5330,-918.878
16,orders - seller offer,562.50,564.86,0.64%,462.86,452.00,-2.27%,-2.91%,-8.99%,3.17%,0.3197,-13.462
18,gmv - seller offer,"16,131.07","16,136.50",0.70%,"13,924.57","13,438.71",-3.80%,-4.50%,-16.84%,7.84%,0.4361,-626.525
19,orders - buy now,"1,864.71","1,821.43",-2.24%,"1,939.43","1,904.00",-1.63%,0.61%,-3.10%,4.32%,0.7264,11.833
25,gmv - buy now,"71,548.99","71,123.29",-0.38%,"82,564.60","84,513.97",3.42%,3.80%,-4.05%,11.65%,0.3133,"3,140.915"
38,gmv_buy_now_200andmore,"14,421.87","14,619.85",4.64%,"19,204.69","23,141.91",23.81%,19.17%,-12.44%,50.78%,0.2136,"3,681.947"


In [27]:
from pmutils.knowledge import add_knowledge, NOTEBOOKNAME

add_knowledge( filename=NOTEBOOKNAME, # doesn't need to be changed
    authors = ['audrey'], # optional, defaults to Analytics
    extra_tags = ['product', 'abtest'], # optional, defaults to None
    summary = 'Affirm Buy Now AB Test', # optional, defaults to 'Click here for details',
    #image_assets=['']
)